In [9]:
from prediction import toolbox, utils
from os import path as osp
from prediction.toolbox import BaseFeatureExtractor

subjects_filename = osp.join('shapes4.json')
subjects_data = toolbox.load_subjects_data_from_json_file(subjects_filename)

successfully loaded 4 subjects from shapes4.json


In [10]:
from collections import defaultdict

# dir(set())

# ?set.remove()



In [1]:
14/16

0.875

In [11]:
import numpy as np
from itertools import combinations
from collections import Counter, defaultdict
from pandas import DataFrame
rnames = ['Right', 'ContainerRight', 'ContainerRight-','ContainerTopRight','ContainerTopRight-','TopRight', 'ContainerMiddleRight']
lnames = ['Left', 'ContainerLeft', 'ContainerLeft-','ContainerTopLeft','ContainerTopLeft-', 'TopLeft', 'ContainerMiddleLeft']
mnames = ['Middle', 'ContainerMiddle-', 'ContainerMiddle']

triple_list = ['Chapter_Shapes-CubeFlatNoGender-ba2351ee-e850-41f7-8f09-29d56ad393f7',
               'Chapter_Shapes-CubeFlatFemale-296ecfd2-80ee-4be6-9ddb-40361c0a5a86',
              'Chapter_Shapes-CylinderSlopeNoGender-68d93470-2675-40a4-b5e3-45e8699deaeb',
              'Chapter_Shapes-CubePeakFemale-65f500ad-d5e2-402d-9e82-203c88862a4a']


def get_1st_sol_time(selected_objs, n):
    """ Calculates the time of the first entered solution.
        selected_objs is a list with objects of the form [uid, container, time]
    """
    uidcont = {}
    containers = {}
    time = 0.0
    for obj in selected_objs:
        cont = uidcont.pop(obj[0], None)
        if cont:
            # object was already in a container. removing
            containers.pop(cont, None)
            # Adding object to new container
            uidcont[obj[0]] = obj[1]
            containers[cont] = obj[0]
        else:
            # object was not in any container
            old_uid = containers.pop(obj[1], None)
            if old_uid:
                # other shape was in the container. removing it
                uidcont.pop(old_uid, None)
            # adding shape to container
            uidcont[obj[0]] = obj[1]
            containers[obj[1]] = obj[0]
        time = obj[2]
        if len(containers.keys()) == n:
            return time
    return None


def which_dir(shp):
    sol = None
    if shp in rnames:
        sol = 'r'
    elif shp in lnames:
        sol = 'l'
    elif shp in mnames:
        sol = 'm'
    return sol


def full_shape(shapes, n):
    count, solution_time = 0, -1
    for s1, s2 in zip(shapes, shapes[1:]):
        if which_dir(s1[0]) != which_dir(s2[0]):
            count += 1
            solution_time = s2[1]
        elif which_dir(s1[0]) == which_dir(s2[0]) and count < n:
            pass
        if count == n:
            return solution_time
        else:
            solution_time = -1
    return solution_time


#is_right_container
def is_right(s1):
    right = False
    if s1 in rnames:
        right = True
    else:
        right = False
    return right

#is_left_container
def is_left(s1):
    left = False
    if s1 in lnames:
        left = True
    else:
        left = False
    return left

#is_middle_container
def is_middle(s1):
    middle = False
    if s1 in mnames:
        middle = True
    else:
        middle = False
    return middle

#check_right_left
def is_there2(shapes):
    newshp, firstr, firstl, max1  = [], True, True, 0.0
    for shp in shapes:
        if is_right(shp[0]):
            if firstr:
                firstr = False
                newshp.insert(0, shp)
        elif is_left(shp[0]):
            if firstl:
                firstl = False
                newshp.insert(1, shp)
    if firstr == False and firstl == False:
        for t in newshp:
            if t[1] > max1:
                max1 = t[1]
    return max1

#check_right_middle_left
def is_there3(shapes):
    newshp, firstr, firstl, firstm, max1  = [], True, True, True, 0.0
    for shp in shapes:
        if is_right(shp[0]):
            if firstr:
                firstr = False
                newshp.insert(0, shp)
        elif is_left(shp[0]):
            if firstl:
                firstl = False
                newshp.insert(1, shp)
        elif is_middle(shp[0]):
            if firstm:
                firstm = False
                newshp.insert(2, shp)
    if firstr == False and firstl == False and  firstm == False:
        for i in newshp:
            if i[1] > max1:
                max1 = i[1]
    return max1

#check_unique_shapes
def unique_l(newshp):
    unique, used, state = [], [], False
    for x in newshp:
        if x[0] not in used:
            used.append(x[0])
            unique.append(x)
    if len(unique) == 4:
        state = True
    return state

#checks_right_left_x2
def is_there4(shapes):
    firstr, firstl, newshp, max1 = True, True, [], 0.0 
    for shp in shapes:
        if is_right(shp[0]) and not unique_l(newshp):
            if firstr:
                firstr = False
                newshp.insert(0, shp)
            else:
                newshp.insert(1, shp)
        elif is_left(shp[0]) and not unique_l(newshp):
            if firstl:
                firstl = False
                newshp.insert(2, shp)
            else:
                newshp.insert(3, shp)
        elif unique_l(newshp):
            break

    if firstr == False and firstl == False and len(newshp) > 3:
        for i in newshp:
            if i[1] > max1:
                max1 = i[1]
        return max1

#check_middle
def middle_chk(uid):
    s = False
    for x in triple_list:
        if x == uid:
            s = True
        else:
            s = False
    return s


def get_dfs(subjects_data):
    dfs = ShapesRnd1Solution().extract(subjects_data)
    return dfs
def get_dfd(subjects_data):    
    dfd = ShapesRnd1SolutionToDone().extract(subjects_data)
    return dfd
def get_dfc(subjects_data):
    dfc = RoundT1stSolutionToLastSelect().extract(subjects_data)
    return dfc


In [12]:
#added
class ShapesRnd1Solution(BaseFeatureExtractor):
    """time from round  start time untill the 1st solution"""
    name = '1st_solution'
    column_prefix = 'r_t1stSolution'
    @staticmethod
    def extract(subjects):
        def get_t1stSolution(rnd):
            sol, shapes, shape_l1 = [], [], []
            try:
                rnd_start = rnd['RoundStartTime']
                rnd_num =int(rnd['RoundName'])
                shape_list = rnd['SelectedShapesList']
                for shape in shape_list:
                    shape_time = shape['SelectTime']
                    shape_cont = shape['ContainerName']
                    shape_l1.append([shape_cont, shape_time])              
                if rnd_num < 4:
                    for shp in shape_l1:
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)
                    if is_there2(shapes) > 0:
                        sol = is_there2(shapes) - rnd_start
            
                if rnd_num > 3  and rnd_num < 8:
                    for shp in shape_l1:                    
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)
                        elif is_middle(shp[0]):
                            shapes.append(shp)
                    if is_there3(shapes) > 0:
                        sol = is_there3(shapes) - rnd_start
                 
                elif rnd_num > 7:
                    for shp in shape_l1:                    
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)                           
                    if is_there4(shapes) > 0:
                        sol = is_there4(shapes) - rnd_start
                
            except TypeError:
                pass
            
            if sol == []:
                sol = np.NaN
            
            return sol
            sol.append()

        dfs = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_t1stSolution, subjects)
        return dfs
ShapesRnd1Solution().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,35.398316,58.909534,40.219590,32.002150,38.59240,44.341680,75.01741,60.71643,87.36902,49.44787,NaN,NaN
5BBA15B120AB8A117956A921,25.764222,45.316052,24.183351,29.251737,43.11293,76.593010,60.33548,65.94654,NaN,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,24.026233,51.938543,36.222756,44.019959,29.93537,34.704330,47.88476,32.14651,82.45452,75.71969,NaN,NaN
5BBCB82A3EF34C681ABFDE1E,42.264637,NaN,47.359043,21.177300,20.60694,19.066711,41.05803,33.06372,43.15837,66.01100,33.48547,53.33372


In [13]:
#added
class ShapesRnd1SolutionToDone(BaseFeatureExtractor):
    """time from 1st solution (all container filled) to hitting done"""
    name = '1st_solution_to_done'
    column_prefix = 'r_t1stSolutionToDone'
    @staticmethod
    def extract(subjects):
        def get_t1stSolutionToDone(rnd):
            sol, shapes, shape_l1 = [], [], []
            rnd_end = rnd['RoundEndTime']
            rnd_num = int(rnd['RoundName'])
            shape_list = rnd['SelectedShapesList']
            for shape in shape_list:
                shape_time = shape['SelectTime']
                shape_cont = shape['ContainerName']
                shape_l1.append([shape_cont, shape_time])              
            if rnd_num < 4:
                for shp in shape_l1:
                    if which_dir(shp[0]) == 'r' or  which_dir(shp[0]) == 'l':
                        shapes.append(shp)
                if full_shape(shapes, 1) > 0:
                    sol = rnd_end - full_shape(shapes, 1)

            if rnd_num > 3  and rnd_num < 8:
                for shp in shape_l1:
                    if which_dir(shp[0]) == 'r' or which_dir(shp[0]) == 'l' or which_dir(shp[0]) == 'm':
                        shapes.append(shp)
                if full_shape(shapes, 2) > 0:
                    sol = rnd_end - full_shape(shapes, 2)

            elif rnd_num > 7:
                for shp in shape_l1:                    
                    if is_right(shp[0]):
                        shapes.append(shp)
                    elif is_left(shp[0]):
                        shapes.append(shp)
                if is_there4(shapes):
                    sol = rnd_end - is_there4(shapes)
            
            if sol == []:
                sol = np.NaN

            return sol
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_t1stSolutionToDone, subjects)
        return df
ShapesRnd1SolutionToDone().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,1.842847,49.168650,1.374000,1.228810,1.12879,1.005610,21.94885,1.28515,1.01599,0.88334,NaN,NaN
5BBA15B120AB8A117956A921,1.498108,0.982548,33.195149,1.115905,14.93100,125.701700,20.94329,6.39026,NaN,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,3.697600,8.768500,17.481000,0.770450,0.72534,0.748000,7.11463,1.01657,1.70886,22.39538,NaN,NaN
5BBCB82A3EF34C681ABFDE1E,1.485300,NaN,1.430725,1.205843,6.68960,1.173449,1.29590,7.32825,1.03926,1.32928,1.1294,1.08291


In [14]:
#added-  code review
class Round1stSolutionToLastSelect(BaseFeatureExtractor):
    """time from 1st solution (all container filled) to last container fill"""
    name = 'r_1stSolutionToLastSelect'
    @staticmethod
    def extract(subjects):
        def get_1stSolutionToLastSelect(rnd):
            sol, uni_shapes, shape_l1, last = [], [], [], 0.0
            rnd_end = rnd['RoundEndTime']
            rnd_num =int(rnd['RoundName'])
            shape_list = rnd['SelectedShapesList']
            for shape in shape_list:
                shape_time = shape['SelectTime']
                shape_cont = shape['ContainerName']
                shape_uid = shape['Shape']['uid']
                shape_l1.append([shape_uid, shape_cont, shape_time])
            last = shape_l1[-1][2]
            if not get_1st_sol_time(shape_l1, 2) or not get_1st_sol_time(shape_l1, 3) or get_1st_sol_time(shape_l1, 4) or not last:
                return np.nan
            if rnd_num < 4 :
                sol = last - get_1st_sol_time(shape_l1, 2)               
            elif 3 < rnd_num < 8 :
                sol = last -  get_1st_sol_time(shape_l1, 3)            
            elif rnd_num > 7:
                sol = last - get_1st_sol_time(shape_l1, 4)
            
            if sol == []:
                sol = np.NaN

            return sol
     
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_1stSolutionToLastSelect, subjects)
        return df
Round1stSolutionToLastSelect().extract(subjects_data)

TypeError: ("unsupported operand type(s) for -: 'float' and 'NoneType'", 'occurred at index 8')

In [ ]:
    

# class RoundT1stSolutionToLastSelectExtractor(BasePreFeatureExtractor):
#     """time from 1st solution (all container filled) to last container fill"""
#     name = 'r_t1stSolutionToLastSelect'   # TODO code review
#
#     @staticmethod
#     def extract(subjects):
#         def get_t_1st_solution_to_last_select(rnd):
#             sol, shapes, shape_l1, last = [], [], [], 0.0
#             try:
#                 rnd_num = int(rnd['RoundName'])
#                 shape_list = rnd['SelectedShapesList']
#                 for shape in shape_list:
#                     shape_time = shape['SelectTime']
#                     shape_cont = shape['ContainerName']
#                     shape_l1.append([shape_cont, shape_time])
#                 if rnd_num < 4:
#                     for shp in shape_l1:
#                         if is_right(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                         elif is_left(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                     if is_there2(shapes) > 0:
#                         sol = last - is_there2(shapes)
#
#                 if 3 < rnd_num < 8:
#                     for shp in shape_l1:
#                         if is_right(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                         elif is_left(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                         elif is_middle(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                     if is_there3(shapes) > 0:
#                         sol = last - is_there3(shapes)
#
#                 elif rnd_num > 7:
#                     for shp in shape_l1:
#                         if is_right(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                         elif is_left(shp[0]):
#                             shapes.append(shp)
#                             last = shp[1]
#                     if is_there4(shapes) > 0:
#                         sol = last - is_there4(shapes)
#             except TypeError:
#                 pass
#
#             if not sol:
#                 sol = np.nan
#
#             return sol
#
#         df = toolbox.create_subjects_data_items_multi_dim_df('shapes', 'Rounds',
#                                                              subjects, get_t_1st_solution_to_last_select)
#         return df


In [6]:
#added
class ShapesRndTargetHeld(BaseFeatureExtractor):
    """number of times target shape is held per round"""
    name = 'r_nTargetHeld'
    @staticmethod
    def extract(subjects):
        def get_nTargetHeld(rnd):
            sol, held_l1, agg = [], [], 0
            try:
                target = rnd['ReferenceShape']
                tar_name = target['name']
                tar_uid = target['uid']
                rnd_num =int(rnd['RoundName'])
                held_list = rnd['ObjectsHeldList']
                for held in held_list:
                    held_l1.append([rnd_num, held['name'], held['uid']])
            except TypeError:
                pass
            
            for h in held_l1:
                if h[2] == tar_uid:
                    agg += 1
            sol = agg
            
            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_nTargetHeld, subjects)
        df.fillna(0, inplace=True)
        return df
ShapesRndTargetHeld().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,1,4,2,1,2,2,3,1,4,3.0,0.0,0.0
5BBA15B120AB8A117956A921,2,2,5,1,3,7,5,2,4,0.0,0.0,0.0
5BBB5F12DC69BC3FA50FA408,0,2,3,3,3,1,4,1,4,2.0,3.0,0.0
5BBCB82A3EF34C681ABFDE1E,1,1,3,2,2,1,3,0,2,4.0,2.0,3.0


In [7]:
#added
class ShapesRndGrabShp(BaseFeatureExtractor):
    """number of times grabbing the same shape in a round"""
    name = 'number_of_times_grab_shape'
    column_prefix = 'r_nGrabShp'
    @staticmethod
    def extract(subjects):
        def get_nGrabShp(rnd):
            grabbed = []
            try:
                grab_l = rnd['ObjectsHeldList']
                for g in grab_l:
                    grabbed.append(g['name'])
                sol = Counter(grabbed)
                return sol
                sol.append
            except TypeError:
                pass
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_nGrabShp, subjects)
        df.fillna(0, inplace=True)
        return df
ShapesRndGrabShp().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,"{'AnswerEasy(Clone)': 1, 'ConeValleyNoGender':...","{'HemisphereValleyNoGender': 1, 'ConeValleyNoG...","{'AnswerEasy2(Clone)': 2, 'PyramidPeakFemale':...","{'AnswerEasy3(Clone)': 1, 'PyramidPeakFemale':...","{'AnswerMedium(Clone)': 2, 'CubeSlopeMale': 2,...","{'AnswerMedium1(Clone)': 2, 'CylinderValleyNoG...","{'PyramidFlatNoGender': 1, 'CylinderSlopeMale1...","{'CubePeakFemale': 2, 'CylinderValleyMale': 1,...","{'AnswersHard(Clone)': 4, 'CubePeakFemale': 1,...","{'TaperedPeakFemale': 1, 'TaperedSlopeNoGender...",0,0
5BBA15B120AB8A117956A921,"{'ConeValleyNoGender': 3, 'AnswerEasy(Clone)':...","{'ConeValleyNoGender': 2, 'ConeValleyFemale': ...","{'AnswerEasy2(Clone)': 5, 'ConeValleyNoGender'...","{'CylinderValleyMale': 1, 'PyramidPeakFemale':...","{'CubeSlopeMale': 1, 'CubeSlopFemale': 1, 'Ans...","{'CylinderValleyMale': 1, 'CubeFlatFemale': 1,...","{'CylinderSlopeMale2': 1, 'AnswerMedium2(Clone...","{'CylinderValleyMale': 1, 'TaperedPeakMale': 1...","{'HemisphereFlatMale': 1, 'CubePeakFemale': 1,...",0,0,0
5BBB5F12DC69BC3FA50FA408,"{'HemisphereValleyFemale': 1, 'ConePeakNoGende...","{'ConeSlopeMale': 1, 'HemisphereValleyFemale':...","{'PyramidPeakNoGender': 1, 'ConeValleyMale': 1...","{'PyramidFlatFemale': 1, 'CylinderFlatNoGender...","{'CubeSlopFemale': 1, 'CubeSlopeMale': 1, 'Ans...","{'HemisphereFlatNoGender': 1, 'CubeFlatNoGende...","{'HemisphereSlopeNoGender': 1, 'CylinderSlopeN...","{'CubePeakFemale': 2, 'CylinderFlatNoGender': ...","{'HemisphereFlatNoGender': 2, 'CubePeakFemale'...","{'ConeValleyMale': 1, 'ConeValleyNoGender': 1,...","{'CubeFlatFemale': 1, 'CylinderSlopeMale2': 1,...",0
5BBCB82A3EF34C681ABFDE1E,"{'ConeStarNoGender': 1, 'ConePeakNoGender': 1,...","{'HemisphereFlatNoGender': 1, 'AnswerEasy1(Clo...","{'PyramidFlatNoGender': 1, 'ConeValleyNoGender...","{'AnswerEasy3(Clone)': 2, 'CubeSlopFemale': 1,...","{'CubeFlatFemale': 2, 'CubeSlopeMale': 1, 'Hem...","{'HemisphereFlatNoGender': 1, 'CylinderValleyN...","{'CylinderSlopeNoGender': 1, 'HemisphereFlatNo...","{'CylinderValleyNoGender': 1, 'CubePeakMale': ...","{'HemisphereFlatNoGender': 1, 'CubePeakFemale'...","{'TaperedPeakNoGender': 1, 'TaperedSlopeNoGend...","{'TaperedPeakFemale': 2, 'CubeFlatNoGender': 1...","{'TaperedPeakFemale': 1, 'TaperedPeakBothGende..."


In [8]:
#added
class ShapesNumberHeldTargetPre1stSelect(BaseFeatureExtractor):
    """number of times holding target pre 1st selection"""
    name = 'number_times_hold_target_pre_1st_selection'
    column_prefix = 'r_nHeldTargetPre1stSelect'
    @staticmethod
    def extract(subjects):
        def get_HeldTargetPre1stSelect(rnd):
            count, select_time = 0, 0.0
            try:
                held_l = rnd['ObjectsHeldList']
                ref_shape = rnd['ReferenceShape']['uid']
                ref_name = rnd['ReferenceShape']['name']
                select_l = rnd['SelectedShapesList']
                for slct in select_l:
                    select_time = slct['SelectTime']
                    break
                for h in held_l:
                    if h['startTime'] < select_time and (h['uid'] == ref_shape or h['name'].find(ref_name) > -1) :
                        count += 1

                return count
                count.append
            except TypeError:
                pass

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_HeldTargetPre1stSelect, subjects)
        #df.fillna(0, inplace=True)
        return df
ShapesNumberHeldTargetPre1stSelect().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,1,1,1,1,1,1,1,1,2,1.0,NaN,NaN
5BBA15B120AB8A117956A921,1,0,2,1,1,1,1,0,2,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,0,2,1,2,1,0,1,0,1,0.0,1.0,NaN
5BBCB82A3EF34C681ABFDE1E,1,1,2,1,0,0,0,0,2,1.0,1.0,1.0


In [9]:
#added
class ShapesTimeHeldTargetPre1stSelect(BaseFeatureExtractor):
    """time holding target pre 1st selection"""
    name = 'r_tHeldTargetPre1stSelect'
    @staticmethod
    def extract(subjects):
        def get_timeHeldTargetPre1stSelect(rnd):
            sol, select_time = 0.0, 0.0
            try:
                held_l = rnd['ObjectsHeldList']
                ref_shape = rnd['ReferenceShape']['uid']
                ref_name = rnd['ReferenceShape']['name']
                select_l = rnd['SelectedShapesList']
                for slct in select_l:
                    select_time = slct['SelectTime']
                    break
                for h in held_l:
                    if h['startTime'] < select_time and (h['uid'] == ref_shape or h['name'].find(ref_name) > -1) :
                        sol += h['length']

                return sol
            except TypeError:
                pass
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_timeHeldTargetPre1stSelect, subjects)
        return df
ShapesTimeHeldTargetPre1stSelect().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,6.345947,6.712341,8.600281,8.097534,5.878052,9.224976,13.792847,14.064575,16.361084,9.943237,NaN,NaN
5BBA15B120AB8A117956A921,3.463135,0.000000,5.196777,3.439880,5.865723,2.122681,1.853271,0.000000,10.084717,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,0.000000,10.466248,6.567810,7.829651,3.529907,0.000000,3.518555,0.000000,14.575684,0.000000,12.945679,NaN
5BBCB82A3EF34C681ABFDE1E,2.893555,2.914185,9.192139,4.491150,0.000000,0.000000,0.000000,0.000000,16.200195,10.303345,6.254883,9.74353


In [10]:
#added
class ShapesGazeAtTargetPre1stHeld(BaseFeatureExtractor):
    """time spent gazing at the target prior 1st grab"""
    name = 'r_tGazeAtTargetPre1stHeld'
    @staticmethod
    def extract(subjects):
        ans_set = []
        def get_GazeAtTargetPre1stHeld(rnd):
            agg = 0
            try:
                rnd_ans = rnd['ReferenceShape']['uid']
                held_l = rnd['ObjectsHeldList']
                for h in held_l:
                    held_1st_t = h['startTime']
                gazed_obj = rnd['ObjectsGazedAtList']
                for g in gazed_obj:
                    gazed_uid = g['uid']
                    gazed_t = g['startTime']
                    gazed_len = g['length']
                    if gazed_uid == rnd_ans and  gazed_t < held_1st_t:
                        agg += gazed_len
                sol = agg
                return sol
            except TypeError:
                pass
                
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_GazeAtTargetPre1stHeld, subjects)
        #df.fillna(0, inplace=True)
        return df
ShapesGazeAtTargetPre1stHeld().extract(subjects_data)
            

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,1.631103,4.266724,4.599426,2.089111,3.906006,3.554443,8.121582,7.441650,8.609619,3.541138,NaN,NaN
5BBA15B120AB8A117956A921,0.000000,0.848328,1.272766,0.000000,1.349060,2.423706,1.695801,4.134155,2.713135,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,0.000000,0.580689,4.983154,0.000000,1.663696,2.513794,3.228272,1.641113,1.038452,20.001587,9.941772,NaN
5BBCB82A3EF34C681ABFDE1E,0.000000,0.525085,0.000000,0.000000,1.218018,0.838928,1.427795,0.659302,1.875854,2.221680,4.007080,0.324219


In [11]:
class ShapesRoundTGazeSolution(BaseFeatureExtractor):
    """time gazing on the target from 1st solution untill end"""
    name = 'time_gaze_solution'
    column_prefix = 'r_tGazeSolution'
    @staticmethod
    def extract(subjects):
        def get_r_TGazeSolution(rnd):
            sol, shapes, shape_l1,gaze_l1, sol_1st, agg = [], [], [], [], 0.0, 0.0
            try:
                rnd_end = rnd['RoundEndTime']
                rnd_num =int(rnd['RoundName'])
                gaze_list = rnd['ObjectsGazedAtList']
                for gaze in gaze_list:
                    gaze_start = gaze['startTime']
                    gaze_name =  gaze['name']
                    gaze_len = gaze['length']
                    gaze_l1.append([gaze_start, gaze_name, gaze_len])
                shape_list = rnd['SelectedShapesList']
                for shape in shape_list:
                    shape_time = shape['SelectTime']
                    shape_cont = shape['ContainerName']
                    shape_l1.append([shape_cont, shape_time])              
                if rnd_num < 4:
                    for shp in shape_l1:
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)
                    if is_there2(shapes) > 0:
                        sol_1st = is_there2(shapes)
            
                if rnd_num > 3  and rnd_num < 8:
                    for shp in shape_l1:                    
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)
                        elif is_middle(shp[0]):
                            shapes.append(shp)
                    if is_there3(shapes) > 0:
                        sol_1st = is_there3(shapes)
                 
                elif rnd_num > 7:
                    for shp in shape_l1:                    
                        if is_right(shp[0]):
                            shapes.append(shp)
                        elif is_left(shp[0]):
                            shapes.append(shp)
                        elif is_tright(shp[0]):
                            shapes.append(shp)
                        elif is_tleft(shp[0]):
                            shapes.append(shp)
                    if is_there4(shapes) > 0:
                        sol_1st = is_there4(shapes)
                for g in gaze_l1:
                    if g[0] > sol_1st and 'Answer' in g[1]:
                        agg += g[2]
                                        
            except TypeError:
                pass
            sol = agg

            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_r_TGazeSolution, subjects)
        return df

ShapesRoundTGazeSolution().extract(subjects_data)


,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,0.0,0.468811,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,NaN,NaN
5BBA15B120AB8A117956A921,0.0,0.000000,0.491211,0.0,0.000000,1.687012,0.0,0.0,0.0,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,0.0,0.580689,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,2.747437,0.0,NaN
5BBCB82A3EF34C681ABFDE1E,0.0,0.525085,0.000000,0.0,0.402222,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [18]:
#added
#irrelevant items list - validate if there are any more irrelevant items
class ShapesTGazeIrrelevant(BaseFeatureExtractor):
    """time gazing at irrelevant things (i.e. walls, windows etc.), or eyes closed"""
    name = 'time_gaze_solution'
    column_prefix = 'r_tGazeIrrelevant'
    @staticmethod
    def extract(subjects):
        def get_TGazeIrrelevant(rnd):
            irr_tot = 0.0
            irrelevant_items = ['Plane Top Shadow', 'Tiles', 'Logo']
            gaze_list = rnd['ObjectsGazedAtList']
            for g in gaze_list:
                gaze_name = g['name']
                gaze_len = g['length']
                if gaze_name in irrelevant_items:
                    irr_tot += gaze_len
                                   
            return irr_tot
            irr_tot.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_TGazeIrrelevant, subjects)
        #df.fillna(0, inplace=True)
        return df.sum(axis=1).to_frame()

ShapesTGazeIrrelevant().extract(subjects_data)

,0
id,
5BB9B96320AB8A117956A917,74.867920
5BBA15B120AB8A117956A921,209.497497
5BBB5F12DC69BC3FA50FA408,119.883789
5BBCB82A3EF34C681ABFDE1E,150.697998


In [23]:
#added
class ShapesNumHeadToClock(BaseFeatureExtractor):
    """number of times directed head at the clock during the entire task"""
    name = 't_nHeadToClock'
    @staticmethod
    def extract(subjects):
        def get_t_nHeadToClock(rnd):
            num = 0
            head_list = rnd.get('ObjectsCenterViewList', [])
            for head in head_list:
                if head['name'] == 'TimerPanel' or head['name'] == 'Timer':
                    num += 1
            return num
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_t_nHeadToClock, subjects)
        return df.sum(axis=1).to_frame()

ShapesNumHeadToClock().extract(subjects_data)

,0
id,
5BB9B96320AB8A117956A917,0.0
5BBA15B120AB8A117956A921,3.0
5BBB5F12DC69BC3FA50FA408,8.0
5BBCB82A3EF34C681ABFDE1E,5.0


In [27]:
#added
class ShapesTime1stHeadToClock(BaseFeatureExtractor):
    """time of first head directed at clock (from task onset)"""
    name = 't_t1stHeadToClock'
    column_prefix = 't_t1stHeadToClock'
    @staticmethod
    def extract(subjects):
        def get_t_t1stHeadToClock(subject):
            data_items = subject['Task_ShapeMatch']['DataItems']
            rounds = data_items['Rounds']
            for st in rounds:
                start = st['RoundStartTime']
                break
            for head in rounds:
                for g in head.get('ObjectsCenterViewList', []):
                    if g['name'] == 'TimerPanel' or g['name'] == 'Timer':
                        return g['startTime'] - start
                return np.nan

        df = toolbox.to_dataframe(subjects_data, get_t_t1stHeadToClock)
        return df
ShapesTime1stHeadToClock().extract(subjects_data)

,0
id,
5BB9B96320AB8A117956A917,NaN
5BBA15B120AB8A117956A921,1.138767
5BBB5F12DC69BC3FA50FA408,NaN
5BBCB82A3EF34C681ABFDE1E,NaN


In [30]:
#added
class ShapesNumGazeToClock(BaseFeatureExtractor):
    """number of times gazed the clock during the entire task"""
    name = 't_nGazeToClock'
    @staticmethod
    def extract(subjects):
        def get_t_nGazeToClock(rnd):
            gaze_count = 0
            gaze_list = rnd.get('ObjectsGazedAtList', [])
            for gaze in gaze_list:
                if gaze['name'] == 'TimerPanel' or gaze['name'] == 'Timer':
                    gaze_count += 1
            return gaze_count
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_t_nGazeToClock, subjects)
        return df.sum(axis=1).to_frame()

ShapesNumGazeToClock().extract(subjects_data)

,0
id,
5BB9B96320AB8A117956A917,7.0
5BBA15B120AB8A117956A921,0.0
5BBB5F12DC69BC3FA50FA408,3.0
5BBCB82A3EF34C681ABFDE1E,0.0


In [32]:
#added -  code review
class ShapesTime1stGazeToClock(BaseFeatureExtractor):
    """time of first gaze to clock (from task onset)"""
    name = 't_t1stGazeToClock'
    column_prefix = 't_t1stGazeToClock'
    @staticmethod
    def extract(subjects):
        def get_t_t1stGazeToClock(subject):
            data_items = subject['Task_ShapeMatch']['DataItems']
            rounds = data_items['Rounds']
            for st in rounds:
                start = st['RoundStartTime']
                break
            for gaze in rounds:
                for g in gaze['ObjectsGazedAtList']:
                    if g['name'] == 'TimerPanel' or g['name'] == 'Timer':
                        return g['startTime'] - start
                return np.nan

        df = toolbox.to_dataframe(subjects_data, get_t_t1stGazeToClock)
        return df
ShapesTime1stGazeToClock().extract(subjects_data)

,0
id,
5BB9B96320AB8A117956A917,NaN
5BBA15B120AB8A117956A921,NaN
5BBB5F12DC69BC3FA50FA408,NaN
5BBCB82A3EF34C681ABFDE1E,NaN


In [8]:
#added
class ShapeRoundMeanRepeatedShpGrab(BaseFeatureExtractor):
    """count grabs per shape and take the average (e.g. grabbed in a round shpA 5 times, shpB 3 times => nMeanRepeatedGrab = 4)"""
    name = 'r_nMeanRepeatedShpGrab'
    column_prefix = 'r_nMeanRepeatedShpGrab'
    @staticmethod
    def extract(subjects):
        def get_r_nMeanRepeatedShpGrab(rnd):
            grabbed, unicnt, count, _sum = [], [], 0, 0
            held_l = rnd.get('ObjectsHeldList', [])
            for h in held_l:
                grabbed.append(h['uid'])
            unicnt = Counter(grabbed)
            for c in unicnt:
                count += 1
                _sum += unicnt[c]
            if count == 0:
                return np.NaN
            return _sum/count


        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_r_nMeanRepeatedShpGrab, subjects)
        return df
ShapeRoundMeanRepeatedShpGrab().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,1.000000,2.333333,1.600000,1.200000,1.333333,1.125000,1.333333,1.166667,2.000000,1.375000,NaN,NaN
5BBA15B120AB8A117956A921,2.000000,1.333333,1.750000,1.000000,1.428571,2.357143,1.875000,1.444444,1.444444,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,1.000000,1.714286,1.666667,1.500000,1.428571,1.125000,1.714286,1.142857,2.100000,1.545455,1.571429,NaN
5BBCB82A3EF34C681ABFDE1E,1.166667,1.000000,1.600000,1.166667,1.333333,1.000000,1.500000,1.571429,1.250000,1.363636,1.250000,1.222222


In [35]:
#added
class ShapeNumLastSolutionWorseThanPrev(BaseFeatureExtractor):
    """number of rounds in which a correct solution was given but the last one was incorrect"""
    name = 't_nLastSolutionWorseThanPrev'
    column_prefix = 't_nLastSolutionWorseThanPrev'
    @staticmethod
    def extract(subjects):
        def get_t_nLastSolutionWorseThanPrev(rnd):
            sol, wor, rnd_ans, slct, c = False, 0, [], [], 0
            rnd_num = int(rnd['RoundName'])
            selected = rnd.get('SelectedShapesList', [])
            correct_l = rnd['RoundCorrectAnswers']['answers'][-1]['answerSet']
            for cor in correct_l:
                rnd_ans.append(cor['uid'])     #contains each rounds answer
            for s in selected:                 #contains each selected shape
                slct.append(s['Shape']['uid'])
            sel = slct[:-1]
            if rnd_num < 3:                    #2 contained rounds
                for shape in sel:
                    if shape in rnd_ans:
                        c += 1
                if c == 2:
                    sol = True
            elif rnd_num > 3 and rnd_num < 8:  #3 contained rounds
                for shape in sel:
                    if shape in rnd_ans:
                        c += 1
                if c == 3:
                    sol = True
            else:                              #4 contained rounds
                for shape in sel:
                    if shape in rnd_ans:
                        c += 1
                if c == 4:
                    sol = True
            if sol == True:
                wor += 1            
            return wor
                
        df = toolbox.create_subjects_data_items_multi_dim_df(
        'shapes', 'Rounds', get_t_nLastSolutionWorseThanPrev, subjects)
        return df                                    
                
ShapeNumLastSolutionWorseThanPrev().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,10,11
id,,,,,,,,,,,,
5BB9B96320AB8A117956A917,0,0,0,0,0,0,0,0,0,0.0,NaN,NaN
5BBA15B120AB8A117956A921,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN
5BBB5F12DC69BC3FA50FA408,0,0,0,0,0,0,0,0,0,0.0,0.0,NaN
5BBCB82A3EF34C681ABFDE1E,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0


In [ ]:
class ShapesRoundFPostVsPreCompletionDur(BaseFeatureExtractor):
    """tCompletionToDone/t1stSolution"""
    name = 'r_fPostVsPreCompletionDur'
    column_prefix = 'r_fPostVsPreCompletionDur'
    @staticmethod
    def extract(subjects_data):
        def get_r_fPostVsPreCompletionDur(subjects_data):
            ndf1 = get_dfs(subjects_data)
            ndf2 = get_dfc(subjects_data)
            ndf = ndf2 / ndf1
            return ndf
        df = get_r_fPostVsPreCompletionDur(subjects_data)
        return df
ShapesRoundFPostVsPreCompletionDur().extract(subjects_data)